# M2 SETTLING TIME ANALYSIS

In [ ]:
%matplotlib inline
import lsst_efd_client
from astropy.time import Time
from pandas import Timedelta
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
global suptitle_size, axtitle_size, axlabel_size, axticks_size
suptitle_size = 16
axtitle_size = 14
axlabel_size = 12
axticks_size = 12

In [ ]:
async def get_query_df(
    efd_client: lsst_efd_client, topic: str, start_time: Time, end_time: Time
):
    """Function to get the query dataframe from the EFD.

    Args:
        efd_client (lsst_efd_client): EFD client object to use for querying.
        topic (str): EFD topic to query.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
    Returns:
        pandas.DataFrame: DataFrame containing the query results.
    """

    topic_fields = await efd_client.get_fields(topic)
    query = efd_client.build_time_range_query(
        topic, topic_fields, start_time, end_time
    )
    query_df = await efd_client.influx_client.query(query)

    return query_df

In [ ]:
async def find_start_stop_slew(
    efd_client: lsst_efd_client, start_time: Time, end_time: Time, axis: str
):
    """Function to find the start and stop times of slews.

    Args:
        efd_client (lsst_efd_client): EFD client object to use for querying.
        start_time (Time): Start time of the query.
        end_time (Time): End time of the query.
        axis (str): Axis to find the slews for (elevation or azimuth).

    Returns:
        dict: dictionary containing the start and stop times of each slew.
    """

    if axis == "elevation":
        topic = "lsst.sal.MTMount.logevent_elevationInPosition"
    elif axis == "azimuth":
        topic = "lsst.sal.MTMount.logevent_azimuthInPosition"

    el_in_position_df = await get_query_df(
        efd_client=efd_client,
        topic=topic,
        start_time=start_time,
        end_time=end_time,
    )
    el_in_position_df["movement"] = 0

    c = 0
    for index, row in el_in_position_df.iterrows():
        if not row["inPosition"]:
            c += 1
        el_in_position_df.loc[index, "movement"] = c

    start_stop = el_in_position_df.groupby(by=["movement"]).groups
    return start_stop

In [ ]:
def plot_overview(axis: str, df: pd.DataFrame, start_stop: dict, rate: int):
    """Function to plot the overview of slews.

    Args:
        axis (str): Axis to plot the overview for (elevation or azimuth).
        df (pandas.DataFrame): DataFrame containing the query results.
        start_stop (dict): dictionary containing the start and stop times of each slew.
        rate (int): Slewing rate.

    Returns:
        None
    """

    fig = plt.figure(layout="constrained")
    fig.suptitle(
        f"{axis.upper()} OVERVIEW ({rate}% slewing rate)",
        fontsize=suptitle_size,
    )
    ax = fig.add_subplot(111)
    df["actualPosition"].plot(
        ax=ax, marker="+", label="", fontsize=axticks_size
    )
    ax.set_ylabel(f"{axis}, deg", fontsize=axlabel_size)
    set_legend = True
    for key, val in start_stop.items():
        if len(val) < 2:
            continue
        if set_legend:
            ax.axvspan(val[0], val[1], alpha=0.3, label="slews")
            set_legend = False
        else:
            ax.axvspan(val[0], val[1], alpha=0.3)
        ax.annotate(
            key, ((val[0] + (val[1] - val[0]) / 2), ax.get_ylim()[1] / 1.1)
        )

    fig.legend()
    # fig.savefig(f"./plot/{axis}_slews_overview_rate{rate}.png", dpi=150)

In [ ]:
def plot_tangent_ferror(
    axis: str, ferror_groupby: dict, start_stop: dict, rate: int
):
    """Function to plot the tangent force error during the slews.

    Args:
        axis (str): Axis to plot the tangent force error for (elevation or azimuth).
        ferror_groupby (dict): Dictionary containing the tangent force error of during each slew.
        start_stop (dict): Dictionary containing the start and stop times of each slew.
        rate (int): Slewing rate.

    Returns:
        None
    """

    fig = plt.figure(
        layout="constrained", figsize=(10, 2 * len(ferror_groupby))
    )
    subfigs = fig.subfigures(
        nrows=len(ferror_groupby), ncols=1, hspace=0.01, wspace=0.02
    )
    for key, move in ferror_groupby.items():
        subfigs[key - 1].suptitle(
            f"{key}th SLEW ({rate}% {axis} slewing rate)",
            fontsize=suptitle_size,
        )
        subfigs[key - 1].set_facecolor("0.75")
        subfigs[key - 1].set_linewidth(0.2)
        ax = subfigs[key - 1].add_subplot(1, 2, 1)
        move["sum"].plot(label="", ax=ax, fontsize=axticks_size)
        ax.set_ylabel("N", fontsize=axlabel_size)
        ax.set_title("SUM", fontsize=axtitle_size)
        line1 = ax.axvline(start_stop[key][0], ls="--", color="green")
        line2 = ax.axvline(start_stop[key][1], ls="--", color="orange")
        line3 = ax.axvline(
            start_stop[key][1] + Timedelta(seconds=2), ls="--", color="red"
        )
        ax = subfigs[key - 1].add_subplot(1, 2, 2)
        move["weight"].plot(label="", ax=ax, fontsize=axticks_size)
        ax.set_ylabel("N", fontsize=axlabel_size)
        ax.set_title("WEIGHT", fontsize=axtitle_size)
        ax.axvline(start_stop[key][0], ls="--", color="green")
        ax.axvline(start_stop[key][1], ls="--", color="orange")
        ax.axvline(
            start_stop[key][1] + Timedelta(seconds=2), ls="--", color="red"
        )
        if key == 1:
            subfigs[key - 1].legend(
                [line1, line2, line3],
                ["start slew", "stop slew", "stop + 2sec"],
                loc="upper left",
            )

    # fig.savefig(f"./plot/{axis}_slews_ferror_rate{rate}.png", dpi=150)

In [ ]:
def plot_ims(
    axis: str, position_ims_groupby: dict, start_stop: dict, rate: int
):
    """Function to plot the IMS position during the slews.

    Args:
        axis (str): Axis to plot the tangent force error for (elevation or azimuth).
        position_ims_groupby (dict): Dictionary containing the IMS position during the each slew.
        start_stop (dict): Dictionary containing the start and stop times of each slew.
        rate (int): Slewing rate.

    Returns:
        None
    """

    fig = plt.figure(
        layout="constrained", figsize=(14, 4 * len(position_ims_groupby) + 1)
    )
    subfigs = fig.subfigures(
        nrows=len(position_ims_groupby), ncols=1, hspace=0.01, wspace=0.02
    )
    dof = ["x", "y", "z", "xRot", "yRot", "zRot"]
    nrow = 2
    ncols = 3
    for key in position_ims_groupby.keys():

        subfigs[key - 1].suptitle(
            f"{key}th SLEW ({rate}% {axis} slewing rate)",
            fontsize=suptitle_size,
        )
        subfigs[key - 1].set_facecolor("0.75")
        subfigs[key - 1].set_linewidth(0.2)

        for i, d in enumerate(dof):
            ax = subfigs[key - 1].add_subplot(nrow, ncols, i + 1)
            if d in dof[:3]:
                ax = position_ims_groupby[key][d].plot(
                    ax=ax, fontsize=axticks_size
                )
                ax.set_ylabel("$\mu$m", fontsize=axlabel_size)
                ax.set_xticks(ax.get_xticks(), labels=[])
            else:
                ax = position_ims_groupby[key][d].plot(
                    ax=ax, fontsize=axticks_size
                )
                ax.set_ylabel("arcsec", fontsize=axlabel_size)

            ax.set_title(f"{d}-IMS", fontsize=axtitle_size)
            line1 = ax.axvline(start_stop[key][0], ls="--", color="green")
            line2 = ax.axvline(start_stop[key][1], ls="--", color="orange")
            line3 = ax.axvline(
                start_stop[key][1] + Timedelta(seconds=2), ls="--", color="red"
            )

        if key == 1:
            subfigs[key - 1].legend(
                [line1, line2, line3],
                ["start slew", "stop slew", "stop + 2sec"],
                loc="upper left",
            )

    # fig.savefig(f"./plot/{axis}_slews_ims_rate{rate}.png", dpi=150)

In [ ]:
def plot_ferror_statistic(
    axis: str, ferror_groupby: dict, start_stop: dict, rate: int, pad: int
):
    """Function to plot the force error statistics after the settling time.

    Args:
        axis (str): Axis to plot the force error statistics for (elevation or azimuth).
        ferror_groupby (dict): Dictionary containing the tangent force error of during each slew.
        start_stop (dict): Dictionary containing the start and stop times of each slew.
        rate (int): Slewing rate.
        pad (int): Time to evaluate after the settling time.

    Returns:
        None
    """

    fig = plt.figure(layout="constrained", figsize=(8, 4))
    fig.suptitle(
        f"FORCE ERROR STATISTIC FOR {pad.seconds}s AFTER THE SETTLING TIME\n({rate}% {axis} slewing rate)",
        fontsize=suptitle_size,
    )

    ax_sum = fig.add_subplot(121)
    ax_sum.set_title("SUM", fontsize=axtitle_size)
    ax_weight = fig.add_subplot(122)
    ax_weight.set_title("WEIGHT", fontsize=axtitle_size)
    x_sum = [key for key in start_stop.keys()]
    yerr_sum_std = [
        val.loc[start_stop[key][1] + Timedelta(seconds=2) :, "sum"].std()
        for key, val in ferror_groupby.items()
    ]
    yerr_sum_ptv = [
        abs(
            val.loc[start_stop[key][1] + Timedelta(seconds=2) :, "sum"].max()
            - val.loc[start_stop[key][1] + Timedelta(seconds=2) :, "sum"].min()
        )
        for key, val in ferror_groupby.items()
    ]

    line1 = ax_sum.scatter(x_sum, yerr_sum_std, marker="X")
    line2 = ax_sum.scatter(x_sum, yerr_sum_ptv, marker="X")

    ax_sum.set_xlabel(xlabel="ith slew", fontsize=axlabel_size)
    ax_sum.set_ylabel(ylabel="N", fontsize=axlabel_size)
    x_weight = [key for key in start_stop.keys()]

    yerr_weight_std = [
        val.loc[start_stop[key][1] + Timedelta(seconds=2) :, "weight"].std()
        for key, val in ferror_groupby.items()
    ]
    yerr_weight_ptv = [
        abs(
            val.loc[
                start_stop[key][1] + Timedelta(seconds=2) :, "weight"
            ].max()
            - val.loc[
                start_stop[key][1] + Timedelta(seconds=2) :, "weight"
            ].min()
        )
        for key, val in ferror_groupby.items()
    ]

    ax_weight.scatter(x_weight, yerr_weight_std, marker="X")
    ax_weight.scatter(x_weight, yerr_weight_ptv, marker="X")
    ax_weight.set_xlabel(xlabel="ith slew", fontsize=axlabel_size)
    ax_weight.set_ylabel(ylabel="N", fontsize=axlabel_size)

    fig.legend([line1, line2], ["Std dev", "PtV"], loc="upper left")

    # fig.savefig(f"./plot/{axis}_slews_ferror_stat_rate{rate}.png", dpi=150)

In [ ]:
def plot_ims_statistic(
    axis: str,
    position_ims_groupby: dict,
    start_stop: dict,
    rate: int,
    pad: int,
):
    """Function to plot the IMS position statistics after the settling time.

    Args:
        axis (str): Axis to plot the IMS position statistics for (elevation or azimuth).
        position_ims_groupby (dict): Dictionary containing the IMS position during the each slew.
        start_stop (dict): Dictionary containing the start and stop times of each slew.
        rate (int): Slewing rate.
        pad (int): Time to evaluate after the settling time.

    Returns:
        None
    """

    fig = plt.figure(layout="constrained", figsize=(8, 5))
    fig.suptitle(
        f"IMS POSITION STATISTIC FOR {pad.seconds}s AFTER THE SETTLING TIME\n({rate}% {axis} slewing rate)",
        fontsize=suptitle_size,
    )

    nrows, ncols = 2, 3
    dof = ["x", "y", "z", "xRot", "yRot", "zRot"]

    for i, d in enumerate(dof):
        ax = fig.add_subplot(nrows, ncols, i + 1)
        x = [key for key in start_stop.keys()]
        y_std = [
            val.loc[start_stop[key][1] + Timedelta(seconds=2) :, d].std()
            for key, val in position_ims_groupby.items()
        ]
        y_ptv = [
            abs(
                val.loc[start_stop[key][1] + Timedelta(seconds=2) :, d].max()
                - val.loc[start_stop[key][1] :, d].min()
            )
            for key, val in position_ims_groupby.items()
        ]
        line1 = ax.scatter(x, y_std, marker="X", label="Std dev")
        line2 = ax.scatter(x, y_ptv, marker="X", label="PtV")
        ax.set_title(f"{d}-IMS", fontsize=axtitle_size)
        ax.set_xlabel("ith slew", fontsize=axlabel_size)

        if i < 3:
            ax.set_ylabel(ylabel="$\mu$m", fontsize=axlabel_size)
        else:
            ax.set_ylabel(ylabel="arcsec", fontsize=axlabel_size)

    fig.legend([line1, line2], ["Std dev", "PtV"], loc="upper left")
    # fig.savefig(f"./plot/{axis}_slews_ims_stat_rate{rate}.png", dpi=150)

In [ ]:
def plot_hardpoint(
    axial_df: pd.DataFrame,
    movement_df: pd.DataFrame,
    start_stop: dict,
    rate: int,
    axis: str,
):

    fig = plt.figure(layout="constrained", figsize=(8, 6))
    fig.suptitle(
        f"HARDPOINT CORRECTIONS OVERVIEW\n({rate}% {axis} slewing rate)",
        fontsize=suptitle_size,
    )
    ax = fig.add_subplot(111)
    hpc_nonzero = axial_df[
        [
            col
            for col in axial_df.columns
            if "hardpoint" in col
            and axial_df[col].max() != axial_df[col].min()
        ]
    ]
    hpc_zero = axial_df[
        [
            col
            for col in axial_df.columns
            if "hardpoint" in col
            and axial_df[col].max() == axial_df[col].min() == 0.0
        ]
    ]

    hpc_nonzero.plot(
        ax=ax, legend=False, lw=0, ms=0, ylabel="N", fontsize=axticks_size
    )

    line_nonzero = ax.fill_between(
        hpc_nonzero.index,
        hpc_nonzero.min(axis=1),
        hpc_nonzero.max(axis=1),
        alpha=0.3,
        color="green",
    )
    ax.plot(
        hpc_nonzero.index, hpc_nonzero.mean(axis=1), color="green", alpha=0.7
    )

    hpc_zero.rename(
        columns={
            col: f"Hard-point B{n}"
            for col, n in zip(hpc_zero.columns, [6, 16, 26])
        }
    ).plot(ax=ax, lw=0, marker="x", fontsize=axticks_size)

    ax2 = ax.twinx()
    movement_df["actualPosition"].plot(
        ax=ax2,
        lw=2,
        alpha=0.5,
        color="black",
        legend=False,
        zorder=0,
        fontsize=axticks_size,
    )

    line0 = ax2.get_lines()[0]
    set_legend = True

    for key, val in start_stop.items():
        if len(val) < 2:
            continue
        if set_legend:
            line1 = ax.axvspan(val[0], val[1], alpha=0.3)
            line2 = ax.axvspan(
                val[1], val[1] + Timedelta(seconds=2), alpha=0.3, color="red"
            )
            set_legend = False
        else:
            ax.axvspan(val[0], val[1], alpha=0.3)
            ax.axvspan(
                val[1], val[1] + Timedelta(seconds=2), alpha=0.3, color="red"
            )

        max_y = (
            ax.get_ylim()[1]
            if abs(ax.get_ylim()[1]) > abs(ax.get_ylim()[0])
            else ax.get_ylim()[0]
        )

        ax.annotate(key, ((val[0] + (val[1] - val[0]) / 2), max_y / 1.1))

    ax.set_ylabel("N", fontsize=axlabel_size)
    ax2.set_ylabel(f"{axis}, deg", fontsize=axlabel_size)
    fig.legend(
        [line_nonzero, line0, line1, line2],
        ["hp. corr.", axis, "slew", "settling"],
        loc="upper left",
        ncols=2,
    )

    # fig.savefig(f"./plot/{axis}_slews_hardpoint_corr_rate{rate}.png", dpi=150)

In [ ]:
efd_client = lsst_efd_client.EfdClient(efd_name="usdf_efd")

## ELEVATION

In [ ]:
rate = 80
axis = "elevation"
time_windows = {
    20: [
        Time("2023-11-16T03:23:30", scale="utc", format="isot"),
        Time("2023-11-16T03:25:30", scale="utc", format="isot"),
    ],
    30: [
        Time("2023-11-16T03:46:14", scale="utc", format="isot"),
        Time("2023-11-16T03:47:30", scale="utc", format="isot"),
    ],
    40: [
        Time("2023-11-16T06:48:00", scale="utc", format="isot"),
        Time("2023-11-16T06:50:00", scale="utc", format="isot"),
    ],
    60: [
        Time("2023-11-18T23:12:45", scale="utc", format="isot"),
        Time("2023-11-18T23:15:00", scale="utc", format="isot"),
    ],
    80: [
        Time("2023-11-19T00:50:50", scale="utc", format="isot"),
        Time("2023-11-19T00:53:00", scale="utc", format="isot"),
    ],
}

start_time, end_time = time_windows[rate][0], time_windows[rate][1]

start_stop_el = await find_start_stop_slew(
    efd_client=efd_client, start_time=start_time, end_time=end_time, axis=axis
)

### Movement overview

In [ ]:
topic = "lsst.sal.MTMount.elevation"
elevation_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=time_windows[rate][0],
    end_time=time_windows[rate][1],
)

plot_overview(axis=axis, df=elevation_df, start_stop=start_stop_el, rate=rate)

### TANGENT FORCE ERROR

In [ ]:
topic = "lsst.sal.MTM2.forceErrorTangent"
tangent_ferror_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=start_time,
    end_time=end_time,
)

pad = Timedelta(seconds=10)
ferror_groupby = dict()

for key, val in start_stop_el.items():
    if key == 0:
        continue
    ferror_groupby[key] = tangent_ferror_df.loc[val[0] : val[1] + pad]

plot_tangent_ferror(
    axis=axis,
    ferror_groupby=ferror_groupby,
    start_stop=start_stop_el,
    rate=rate,
)

plot_ferror_statistic(
    axis=axis,
    ferror_groupby=ferror_groupby,
    rate=rate,
    start_stop=start_stop_el,
    pad=pad,
)

### RIGID BODY POSITION (IMS)

In [ ]:
topic = "lsst.sal.MTM2.positionIMS"
position_ims_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=start_time,
    end_time=end_time,
)

position_ims_groupby = dict()

for key, val in start_stop_el.items():
    if key == 0:
        continue
    position_ims_groupby[key] = position_ims_df.loc[val[0] : val[1] + pad]

plot_ims(
    axis=axis,
    position_ims_groupby=position_ims_groupby,
    start_stop=start_stop_el,
    rate=rate,
)

plot_ims_statistic(
    axis=axis,
    position_ims_groupby=position_ims_groupby,
    start_stop=start_stop_el,
    rate=rate,
    pad=pad,
)

### HARDPOINT CORRECTION

In [ ]:
topic = "lsst.sal.MTM2.axialForce"
axial_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=time_windows[rate][0],
    end_time=time_windows[rate][1],
)

plot_hardpoint(
    axial_df=axial_df,
    movement_df=elevation_df,
    start_stop=start_stop_el,
    rate=rate,
    axis=axis,
)

## AZIMUTH

In [ ]:
rate = 5
axis = "azimuth"
time_windows = {
    1: [
        Time("2023-11-16T08:26:30", scale="utc", format="isot"),
        Time("2023-11-16T08:33:00", scale="utc", format="isot"),
    ],
    3: [
        Time("2023-11-16T08:43:30", scale="utc", format="isot"),
        Time("2023-11-16T08:47:30", scale="utc", format="isot"),
    ],
    5: [
        Time("2023-11-16T08:57:30", scale="utc", format="isot"),
        Time("2023-11-16T09:00:30", scale="utc", format="isot"),
    ],
}

start_time, end_time = time_windows[rate][0], time_windows[rate][1]

start_stop_az = await find_start_stop_slew(
    efd_client=efd_client, start_time=start_time, end_time=end_time, axis=axis
)

### MOVEMENT OVERVIEW

In [ ]:
topic = "lsst.sal.MTMount.azimuth"
azimuth_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=time_windows[rate][0],
    end_time=time_windows[rate][1],
)

plot_overview(axis=axis, df=azimuth_df, start_stop=start_stop_az, rate=rate)

### TANGENT FORCE ERROR

In [ ]:
topic = "lsst.sal.MTM2.forceErrorTangent"
tangent_ferror_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=start_time,
    end_time=end_time,
)

pad = Timedelta(seconds=10)
ferror_groupby = dict()

for key, val in start_stop_az.items():
    if key == 0:
        continue
    ferror_groupby[key] = tangent_ferror_df.loc[val[0] : val[1] + pad]

plot_tangent_ferror(
    axis=axis,
    ferror_groupby=ferror_groupby,
    start_stop=start_stop_az,
    rate=rate,
)

plot_ferror_statistic(
    axis=axis,
    ferror_groupby=ferror_groupby,
    rate=rate,
    start_stop=start_stop_az,
    pad=pad,
)

### RIGID BODY POSITION (IMS)

In [ ]:
topic = "lsst.sal.MTM2.positionIMS"
position_ims_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=start_time,
    end_time=end_time,
)

position_ims_groupby = dict()

for key, val in start_stop_az.items():
    if key == 0:
        continue
    position_ims_groupby[key] = position_ims_df.loc[val[0] : val[1] + pad]

plot_ims(
    axis=axis,
    position_ims_groupby=position_ims_groupby,
    start_stop=start_stop_az,
    rate=rate,
)

plot_ims_statistic(
    axis=axis,
    position_ims_groupby=position_ims_groupby,
    start_stop=start_stop_az,
    rate=rate,
    pad=pad,
)

### HARDPOINT CORRECTION

In [ ]:
topic = "lsst.sal.MTM2.axialForce"
axial_df = await get_query_df(
    efd_client=efd_client,
    topic=topic,
    start_time=time_windows[rate][0],
    end_time=time_windows[rate][1],
)

plot_hardpoint(
    axial_df=axial_df,
    movement_df=azimuth_df,
    start_stop=start_stop_az,
    rate=rate,
    axis=axis,
)